In [11]:
import requests
from bs4 import BeautifulSoup

In [12]:
%%html
<iframe frameborder="no" border="0" marginwidth="0" marginheight="0" 
    width=900 height=600 
    src="http://hprc.cssn.cn/wxzl/wxysl/lczf/">
</iframe>

In [13]:
# get the link for each year
url = "http://hprc.cssn.cn/wxzl/wxysl/lczf/" 
content = requests.get(url)
content.encoding

'ISO-8859-1'

In [ ]:
# Specify the encoding
content.encoding = 'utf8' # 'gb18030'
content = content.text

In [17]:
soup = BeautifulSoup(content, 'html.parser') 
# links = soup.find_all('td', {'class', 'bl'})   
links = soup.select('.bl a')
print(links[0])

<a href="./dssjqgrmdbdh_1/202103/t20210316_5318622.html" target="_blank" title="2021骞存斂搴滃伐浣滄姤鍛�">2021骞存斂搴滃伐浣滄姤鍛�</a>


In [18]:
len(links)

52

In [19]:
links[-1]['href']

'./dishiyijie_10/200908/t20090818_3955459.html'

In [20]:
links[0]['href'].split('./')[1]

'dssjqgrmdbdh_1/202103/t20210316_5318622.html'

In [21]:
url + links[0]['href'].split('./')[1]

'http://hprc.cssn.cn/wxzl/wxysl/lczf/dssjqgrmdbdh_1/202103/t20210316_5318622.html'

In [22]:
hyperlinks = [url + i['href'].split('./')[1] for i in links]
hyperlinks[:5]

['http://hprc.cssn.cn/wxzl/wxysl/lczf/dssjqgrmdbdh_1/202103/t20210316_5318622.html',
 'http://hprc.cssn.cn/wxzl/wxysl/lczf/dssjqgrmdbdh_1/202006/t20200604_5138985.html',
 'http://hprc.cssn.cn/wxzl/wxysl/lczf/dssjqgrmdbdh_1/201903/t20190318_4849567.html',
 'http://hprc.cssn.cn/wxzl/wxysl/lczf/dssjqgrmdbdh_1/201803/t20180323_4240852.html',
 'http://hprc.cssn.cn/wxzl/wxysl/lczf/d12qgrdzfbg/201703/t20170317_4144138.html']

In [25]:
import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()
    
def crawler(url_i):
    content = requests.get(url_i)
    content.encoding = 'utf8'  
    content = content.text
    soup = BeautifulSoup(content, 'html.parser') 
    year = soup.find('span', {'class', 'huang16c'}).text[:4]
    year = int(year)
    report = ''.join(s.text for s in soup('p'))
    # 找到分页信息
    scripts = soup.find_all('script')
    countPage = int(''.join(scripts[1]).split('countPage = ')[1].split('//')[0])
    if countPage == 1:
        pass
    else:
        for i in range(1, countPage):
            url_child = url_i.split('.html')[0] +'_'+str(i)+'.html'
            content = requests.get(url_child)
            content.encoding = 'gb18030'
            content = content.text
            soup = BeautifulSoup(content, 'html.parser') 
            report_child = ''.join(s.text for s in soup('p'))
            report = report + report_child
    return year, report

In [26]:
# 抓取52年政府工作报告内容
reports = {}
for link in hyperlinks:
    year, report = crawler(link)
    flushPrint(year)
    reports[year] = report 

1954

In [31]:
with open('/Users/laoke/Desktop/研究生-好好学习/大数据挖掘/作业四：抓取政府工作报告/政府工作报告.txt', 'w', encoding = 'utf8') as f:
    for r in reports:
        line = str(r)+'\t'+reports[r].replace('\n', '\t') +'\n'
        f.write(line)

In [32]:
import pandas as pd

df = pd.read_table('/Users/laoke/Desktop/研究生-好好学习/大数据挖掘/作业四：抓取政府工作报告/政府工作报告.txt', names = ['year', 'report'])

In [29]:
df[:5]

,year,report
0,2021,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...
1,2020,新华社北京5月29日电 政府工作报告 ——2020年5月22日在第十三届全国人民代表...
2,2019,新华社北京3月16日电 政府工作报告 ——2019年3月5日在第十三届全国人民代表...
3,2018,各位代表： 现在，我代表国务院，向大会报告过去五年政府工作，对今年工作提出建议，请予审...
4,2017,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协各位委员提出...


In [30]:
df[:53]

,year,report
0,2021,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。 ...
1,2020,新华社北京5月29日电 政府工作报告 ——2020年5月22日在第十三届全国人民代表...
2,2019,新华社北京3月16日电 政府工作报告 ——2019年3月5日在第十三届全国人民代表...
3,2018,各位代表： 现在，我代表国务院，向大会报告过去五年政府工作，对今年工作提出建议，请予审...
4,2017,各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协各位委员提出...
5,2016,政府工作报告
6,2015,国务院总理 李克强 各位代表： 现在，我代表国务院，向大会报告政府工作，请予审议，并...
7,2014,3月5日，李克强代表国务院在十二届全国人大二次会议上作《政府工作报告》。 新华社记者姚大...
8,2013,各位代表： 现在，我代表国务院，向大会报告过去五年的政府工作，并对今年工作提出建议，请...
9,2012,3月5日，国务院总理温家宝在第十一届全国人民代表大会第五次会议上作政府工作报告。记者 李...
